In [1]:
import os
import numpy as np
import Model

from config import root, dir_dataset_train, dir_dataset_val

import warnings
warnings.filterwarnings("ignore")

In [2]:
model_name = 'v1'
dir_model =  os.path.join(root, 'Model/', model_name)

# model = Model.create_model()
model = Model.load_model(dir_model)

Successfully loaded model from ../Model/v1


In [3]:
# Model.save_model(dir_model, model)

INFO:tensorflow:Assets written to: ../Model/v1\assets
Successfully saved model to ../Model/v1


In [4]:
def create_index(source):
    return (source, os.listdir(source))

def generator(source, index, num_batch):
    while(True):
        np.random.shuffle(index)
        for i in range(num_batch):
            X, y = np.load( os.path.join(source, index[i]), allow_pickle=True )
            X = np.asarray( list(X), 'float32' )
            y = np.asarray( list(y), 'float32' )
            yield(X, y)    

def create_generator(index):
    source = index[0]
    index = index[1]
    num_batch = len(index)
    gen = generator(source, index, num_batch)
    return gen, num_batch

In [5]:
train_index = create_index(dir_dataset_train)
val_index = create_index(dir_dataset_val)

train_gen, train_num_batch = create_generator(train_index)
val_gen, val_num_batch = create_generator(val_index)

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath=dir_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint]

history = model.fit_generator(
    generator = train_gen,
    steps_per_epoch = train_num_batch,
    epochs = 2,
    validation_data = val_gen,
    validation_steps = val_num_batch,
    max_queue_size = 20,
    verbose = 1,
    callbacks = callbacks
)

In [9]:
result = model.evaluate_generator(
    generator = val_gen,
    steps = val_num_batch,
    max_queue_size = 20,
    verbose = 1,
)
print(result)

225/225 [==============================] - 124s 367ms/step - loss: 1.4045 - accuracy: 0.4659
[1.404510259628296, 0.46586793661117554]
